In [1]:
import json

f = open('data/vectorized_lines.json', 'r', encoding='utf-8')
vectorized_lines = json.load(f)
f.close()

print(len(vectorized_lines))
print(vectorized_lines[0])

58488
{'line_text': ['PAinter', ',', 'once', 'more', 'thy', 'Pencil', 'reassume', '.'], 'line_pos': ['n1', '', 'acp', 'avc-d', 'po', 'n1', 'vvi', ''], 'line_arpabet': ['p/ey/n/t/er', '', 'w/ah/n/s', 'm/ao/r', 'dh/ay', 'p/eh/n/s/ah/l', 'r/iy/ah/s/uw/m', ''], 'line_ipa': ['p/eɪ/n/t/ɹ̩', '', 'w/ʌ/n/s', 'm/ɔ/ɹ', 'ð/aɪ', 'p/ɛ/n/s/ʌ/l', 'ɹ/i/ʌ/s/u/m', ''], 'line_hayes_features': ['001000100000000000000000000000/010010001001010000001000101010/100100100000000000101000000010/100100100000000000000000000000/100010001010000000001000100010', '', '011011001000100000011000001010/010011001000000000001000100010/100100100000000000101000000010/100100101100000000000001000000', '001000100000000000101000000010/011011001000000000011000100010/100010001010000000001000000010', '100100101110000000000000000010/010010001000010001001000100010', '001000100000000000000000000000/010010001001000000001000100010/100100100000000000101000000010/100100101100000000000001000000/010011001000000000001000100010/10011010100000010

In [2]:
lines_by_author = {}

for l in vectorized_lines:
    if l['author'] not in lines_by_author:
        lines_by_author[l['author']] = []
    lines_by_author[l['author']].append(l)
    
for author, lines in lines_by_author.items():
    print(author, len(lines))
    
author_names = sorted(list(lines_by_author.keys()))

print()
print(author_names)

Marvell 8418
Vaughan 8893
Donne 13346
Crashaw 8545
Rochester 3875
Herbert 5420
Herrick 9991

['Crashaw', 'Donne', 'Herbert', 'Herrick', 'Marvell', 'Rochester', 'Vaughan']


In [3]:
import numpy as np

def lines_to_features(lines):
    
    all_lines_features = []
    
    for l in lines:
        
        l_features = [0 for a in range(0, 30)]
        
        hayes = '/'.join(l['line_hayes_features']).split('/')
        
        for h in hayes:
            i_features = [int(c) for c in h]
            for a, i in enumerate(i_features):
                l_features[a] += i
                
        all_lines_features.append(l_features)

    return all_lines_features

# -----------------------------------------------------------------------------------

features_by_author = {}

for author, lines in lines_by_author.items():
    features_by_author[author] = lines_to_features(lines)
    
print(len(features_by_author))


7


In [36]:
import random
import numpy as np

from sklearn.naive_bayes import *
from sklearn import tree, svm
from sklearn.linear_model import *
from sklearn.ensemble import *
from sklearn.preprocessing import *
    
# -----------------------------------------------------------------------------------
        
for a in range(0, len(author_names)):
    
    print()
    print('----------------------------------------------')
    print()
    
    for b in range(0, len(author_names)):
        
        if a == b:
            continue
        
        print()
        print(author_names[a], 'vs.', author_names[b])
        print()
        
        a_lines = features_by_author[author_names[a]]
        b_lines = features_by_author[author_names[b]]
        
        random.shuffle(a_lines)
        random.shuffle(b_lines)
        
        min_lines = len(a_lines)
        if len(b_lines) < min_lines:
            min_lines = len(b_lines)
        
        training_data = a_lines[:3000] + b_lines[:3000]
        testing_data = a_lines[3000:min_lines] + b_lines[3000:min_lines]
        
        training_labels = [author_names[a] for l in a_lines[:3000]] + \
                            [author_names[b] for l in b_lines[:3000]]
        testing_labels = [author_names[a] for l in a_lines[3000:min_lines]] + \
                            [author_names[b] for l in b_lines[3000:min_lines]]
        
        #print('\t', 'a_lines', len(a_lines), 'b_lines', len(b_lines))
        #print()
        #print('\t', 'min_lines', min_lines)
        #print('\t', 'training_data', len(training_data))
        #print('\t', 'training_labels', len(training_labels))
        #print('\t', 'testing_data', len(testing_data))
        #print('\t', 'testing_labels', len(testing_labels))
        #print()
        
        # ----------------------------------------------------------------------------

        #clf = BernoulliNB(fit_prior=False).fit(training_data, training_labels)

        #score = clf.score(testing_data, testing_labels)

        #print('\t', 'BernoulliNB False', score)
        
        # ----------------------------------------------------------------------------

        #clf = MultinomialNB(fit_prior=False).fit(training_data, training_labels)

        #score = clf.score(testing_data, testing_labels)

        #print('\t', 'MultinomialNB False', score)
        
        # ----------------------------------------------------------------------------

        #clf = tree.DecisionTreeClassifier()
        #clf = clf.fit(training_data, training_labels)

        #score = clf.score(testing_data, testing_labels)

        #print('\t', 'DecisionTreeClassifier', score)
        
        # ----------------------------------------------------------------------------

        #clf = svm.SVC(kernel='linear')
        #clf = clf.fit(training_data, training_labels)

        #score = clf.score(testing_data, testing_labels)
        
        #print('\t', 'SVC', score)
        
        # ----------------------------------------------------------------------------

        #clf = SGDClassifier()
        #clf = clf.fit(training_data, training_labels)

        #score = clf.score(testing_data, testing_labels)
        
        #print('\t', 'SGDClassifier', score)
        
        # ----------------------------------------------------------------------------

        clf = RandomForestClassifier().fit(training_data, training_labels)
        
        score = clf.score(testing_data, testing_labels)
        
        print('\t', 'RandomForestClassifier', score)
        
        # ----------------------------------------------------------------------------

        #estimators = [
        #    ('rfc', RandomForestClassifier()),
        #    ('dtc', tree.DecisionTreeClassifier()),
        #    ('svc', svm.SVC(kernel='linear', probability=True))]

        #clf = VotingClassifier(estimators=estimators, voting='soft')
        #clf.fit(training_data, training_labels)
        
        #score = clf.score(testing_data, testing_labels)
        
        #print('\t', 'VotingClassifier (1)', score)
        
        # ----------------------------------------------------------------------------

        #estimators = [
        #    ('rfc', RandomForestClassifier()),
        #    ('svc', svm.SVC(kernel='linear', probability=True))]

        #clf = VotingClassifier(estimators=estimators, voting='soft')
        #clf.fit(training_data, training_labels)
        
        #score = clf.score(testing_data, testing_labels)
        
        #print('\t', 'VotingClassifier (2)', score)
        
        # ----------------------------------------------------------------------------
        
print()
print('ok!')


----------------------------------------------


Crashaw vs. Donne

	 RandomForestClassifier 0.7731289449954915

Crashaw vs. Herbert

	 RandomForestClassifier 0.6547520661157025

Crashaw vs. Herrick

	 RandomForestClassifier 0.6404869251577998

Crashaw vs. Marvell

	 RandomForestClassifier 0.6665743816906607

Crashaw vs. Rochester

	 RandomForestClassifier 0.6582857142857143

Crashaw vs. Vaughan

	 RandomForestClassifier 0.6394950405770965

----------------------------------------------


Donne vs. Crashaw

	 RandomForestClassifier 0.7741208295761948

Donne vs. Herbert

	 RandomForestClassifier 0.6942148760330579

Donne vs. Herrick

	 RandomForestClassifier 0.7605492776426834

Donne vs. Marvell

	 RandomForestClassifier 0.6480251015134736

Donne vs. Rochester

	 RandomForestClassifier 0.6497142857142857

Donne vs. Vaughan

	 RandomForestClassifier 0.6948073986085186

----------------------------------------------


Herbert vs. Crashaw

	 RandomForestClassifier 0.6683884297520661

Herb